# Generate Test Results

## Prepare Test Images

In [1]:
from utils.prepare_images import prepare_images
import os
import pandas as pd

%load_ext autoreload
%autoreload 2

In [2]:
# put videos here!
video_base = 'data/videos'
#video_base = 'D:/e6691/6691_assignment2/videos'
videos = [vid for vid in os.listdir(video_base) if vid.endswith('mp4')]
# images will be output to here
image_base = 'data/images'
#image_base = 'D:/e6691/6691_assignment2/images'

In [28]:
test_videos = sorted(videos)[1:2]
test_videos

['RALIHR_surgeon01_fps01_0002.mp4']

In [29]:
# command line: 
# ffmpeg -i {input_video} -r {frame_rate} [-f {force_format} (not needed)] {output_images}
# doc: https://ffmpeg.org/ffmpeg.html
for video in test_videos:
    input_path = os.path.join(video_base, video)
    # make dirs
    output_base = image_base + '/{}'.format(video.split('.')[0])
    if not os.path.exists(output_base):
        os.mkdir(output_base)
    output_path = os.path.join(output_base, '%d.png')
    # # command
    print('Frames extracted from {} to {}'.format(input_path, output_path))
    # !!!!!! BECAREFUL WITH THIS LINE !!!!!!
    #!ffmpeg -i {input_path} -r 1 {output_path}

Frames extracted from data/videos/RALIHR_surgeon01_fps01_0002.mp4 to data/images/RALIHR_surgeon01_fps01_0002/%d.png


In [30]:
labels_path = 'data/labels/video.phase.trainingData.clean.StudentVersion.csv'
names_path = 'data/labels/all_labels_hernia.csv'
labels_df = pd.read_csv(labels_path)
names_df = pd.read_csv(names_path, index_col='id')

In [ ]:
# all labels are set to 14
prepare_images(test_videos, video_base, image_base, labels_df, names_df)

## Predict

In [12]:
from models.SVRCNet.svrc import SVRC
from utils.trainer import Evaluator
from utils.read_videos import read

import torch
from torchvision import transforms

print(torch.version.cuda)

None


In [33]:
image_paths, labels = read(test_videos, image_base, ind_end=60)

In [34]:
X_test = sum(image_paths, [])
y_test = sum(labels, [])

In [ ]:
list(zip(X_test, y_test))

In [36]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = SVRC()
if torch.cuda.is_available():
    model.to(device)

# this does not necessarily affect what mode we're running at
# weights of resnet is not changed anyway
# !!!!!! DELETE THIS LINE !!!!!!
WeightsPath_LSTM = 'models/weights_resnet18_70_LSTM'
model.load_state_dict(torch.load(WeightsPath_LSTM, map_location=device))

<All keys matched successfully>

In [37]:
data_transform = {
    "train": transforms.Compose([
        transforms.Resize((32,32)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485], std=[0.229])
    ])
}

In [38]:
evaluator = Evaluator(model, device)

In [39]:
preds_res = evaluator.predict(X_test, data_transform['train'], pretrain=True)

In [40]:
preds_lstm = evaluator.predict(X_test, data_transform['train'], pretrain=False)

In [ ]:
# seems that lstm does do something in terms of smoothing the predictions
list(zip(preds_res, preds_lstm))

In [56]:
from scipy.signal import medfilt

In [57]:
preds_res_filtered = medfilt(preds_lstm, 3)
preds_res_filtered

array([13, 13, 13, ...,  5,  5,  5])

In [ ]:
evaluator.eval(preds_res, y_test)
evaluator.eval(preds_lstm, y_test)
evaluator.eval(preds_res_filtered, y_test)

## Convert to csv

In [ ]:
from parse import parse

In [50]:
template_path = 'data/labels/kaggle_template.csv'

In [ ]:
template_df = pd.read_csv(template_path, names=['Predicted'])
template_df

In [62]:
# a module to parese image path
#!pip install parse

  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24591 sha256=6d291cbd38081a79135d91dd7611b263ba62ccfb53a69817a0ad675ff790d1a9
  Stored in directory: /Users/harrypotter/Library/Caches/pip/wheels/d6/9c/58/ee3ba36897e890f3ad81e9b730791a153fce20caa4a8a474df
Successfully built parse


In [73]:
parse('{}/RALIHR_surgeon{}_fps01_{}/{}-{}.png', X_test[0])[1:4]

('01', '0002', '1')

In [112]:
def format_path(s):
    ss = parse('{}/RALIHR_surgeon{}_fps01_{}/{}-{}.png', s)[1:4]
    return '{}-{}-{}'.format(
        ss[0].rjust(3, '0'), 
        ss[1].rjust(4, '0'), 
        ss[2].rjust(5, '0')
    )

In [ ]:
names_df.loc[preds_res_filtered].values.flatten()

In [ ]:
test_df = pd.DataFrame({
    'preds': names_df.loc[preds_res_filtered].values.flatten()
}, index=list(map(format_path, X_test)))
test_df

In [ ]:
template_df.index

In [ ]:
test_df.index

In [ ]:
# select test_df with template
preds_df = test_df.loc[test_df.index] #[template_df.index]
preds_df

In [ ]:
preds_df.to_csv('data/labels/kaggle_preds.csv')